In [1]:
# dependencies to run the notebook

# !pip install torch==1.12.1
# !pip install torchmetrics==0.10.2
# !pip install torchvision==0.14.0
# !pip install texttable==1.6.4


In [2]:
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import torch.optim as optim
import warnings
import numpy as np
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import classification_report
from texttable import Texttable
from torch.utils.data.sampler import SubsetRandomSampler

warnings.filterwarnings("ignore")


In [3]:
EMBEDDINGS_NAME = "word2vec"


In [4]:
BASE_PATH = "."
TRAIN_EMBEDDINGS = f"../data/word2vec_train.pkl"
TEST_EMBEDDINGS = f"../data/word2vec_test.pkl"
TRAIN_DATASET_PATH = f"{BASE_PATH}/../legal_bert/data/tos_clauses_train.csv"
TEST_DATASET_PATH = f"{BASE_PATH}/../legal_bert/data/tos_clauses_dev.csv"
RNN_MODEL_PATH = f"{BASE_PATH}/models/rnn_{EMBEDDINGS_NAME}_model.pt"
GRU_MODEL_PATH = f"{BASE_PATH}/models/gru_{EMBEDDINGS_NAME}_model.pt"
LSTM_MODEL_PATH = f"{BASE_PATH}/models/lstm_{EMBEDDINGS_NAME}_model.pt"
BILSTM_MODEL_PATH = f"{BASE_PATH}/models/bilstm_{EMBEDDINGS_NAME}_model.pt"


In [5]:
# Read train BERT embeddings
with open(TRAIN_EMBEDDINGS, "rb") as f:
    training_data = pickle.load(f)


In [6]:
# Read test BERT embeddings
with open(TEST_EMBEDDINGS, "rb") as f:
    testing_data = pickle.load(f)


In [7]:
# # check
# for item in training_data:
#     print("The data is : ", item)


In [8]:
len(training_data), len(training_data[0])

(7531, 300)

<span style="color:darkviolet">
<font size="4">Get the labels from train and test files.</font>
</span>


In [9]:
TRAIN_DATASET_PATH = "../legal_bert/data/tos_clauses_train.csv"
TEST_DATASET_PATH = "../legal_bert/data/tos_clauses_dev.csv"


In [10]:
train_df = pd.read_csv(TRAIN_DATASET_PATH, header=0)
test_df = pd.read_csv(TEST_DATASET_PATH, header=0)


In [11]:
train_targets = train_df.label.values
test_targets = test_df.label.values


In [12]:
test_targets


array([0, 0, 0, ..., 0, 1, 0])

In [13]:
device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
# elif torch.backends.mps.is_available():
#     device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using Device: {device}")


Using Device: cpu


<span style="color:darkviolet">
<font size="4">Create Dataset, Train and Test Classes</font>
</span>


In [14]:
class Dataset(object):
    """An abstract class representing a Dataset.
    All other datasets should subclass it. All subclasses should
    override ``__len__``, that provides the size of the dataset,
    and ``__getitem__``, supporting integer indexing in range
    from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])


In [15]:
class TOSDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.data1 = X
        self.data2 = Y
        self.transform = transform

    def __len__(self):
        return len(self.data1)

    def __getitem__(self, index):
        x = self.data1[index]
        y = self.data2[index]

        if self.transform is not None:
            x = torch.tensor(x)

        return torch.tensor(x), torch.tensor(y)


In [16]:
test_len = len(test_df)
train_len = len(train_df)
X_train_tensor = TOSDataset(train_df["sentences"], train_df["label"])
# X_test_tensor = Train_Model(test_df)

num_train = len(X_train_tensor)
indices = list(range(num_train))
np.random.shuffle(indices)
# split = int(np.floor(num_train))
# train_idx = indices[split:]

train_sampler = SubsetRandomSampler(indices)
# valid_sampler = SubsetRandomSampler(valid_idx)
print(train_sampler)
train_df_by_index = train_df.loc[indices]
# val_df_by_index = df_train.loc[valid_idx]
train_fair = sum(train_df_by_index["label"] == 0)
train_unfair = sum(train_df_by_index["label"] == 1)
# val_fair = sum(val_df_by_index['label'] == 0)
# val_unfair = sum(val_df_by_index['label'] == 1)

print("train_fair:" + str(train_fair))
print("train_unfair:" + str(train_unfair))
# print("val_fair:" + str(val_fair))
# print("val_unfair:" + str(val_unfair))


train_fair:6705
train_unfair:826


In [17]:
train_data = TOSDataset(training_data, train_targets, transform=transforms.ToTensor())
test_data = TOSDataset(testing_data, test_targets, transform=transforms.ToTensor())


<span style="color:darkviolet">
<font size="4">Prepare Data loaders</font>
</span>


In [18]:
# how many samples per batch to load
BATCH_SIZE = 20

# number of subprocesses to use for data loading
NUM_WORKERS = 0


In [19]:
# prepare data loaders
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


In [20]:
# check sizes
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print("Sample input size: ", sample_x.size())  # batch_size, seq_length
# print("Sample input: \n", sample_x)
print()
print("Sample label size: ", sample_y.size())  # batch_size
# print("Sample label: \n", sample_y)


AttributeError: '_SingleProcessDataLoaderIter' object has no attribute 'next'

In [ ]:
torch.squeeze(sample_x, dim=1).shape


<span style="color:darkviolet">
<font size="5">SIMPLE RNN</font><br>
<font size="2.5">Number of hidden dimension : 20</font> <br>
<font size="2.5">Number of layers: 1</font> <br>
<font size="2.5">Number of epochs: 5</font> <br>
</span>


In [21]:
class RNNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(RNNet, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=3, batch_first=True, nonlinearity="relu")

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(3, self.hidden_dim))
        h0 = h0.to(device)
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out)
        return out


In [22]:
import time


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [23]:
EMBEDDING_DIM = 300
OUTPUT_DIM = 2


In [24]:
RNN_HIDDEN_DIM = 20
RNN_N_EPOCHS = 10


In [25]:
from sklearn.utils.class_weight import compute_class_weight

class_weight = compute_class_weight(
    "balanced", classes=np.unique(train_df_by_index["label"]), y=train_df_by_index["label"]
)
class_weight


array([0.56159582, 4.55871671])

In [26]:
model = RNNet(EMBEDDING_DIM, RNN_HIDDEN_DIM, OUTPUT_DIM)
model = model.to(device)
loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight))
optimizer = optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(RNN_N_EPOCHS):

    start_time = time.time()
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        inputs = torch.squeeze(inputs)
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs)
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
        torch.save(model.state_dict(), RNN_MODEL_PATH)
        test_min_loss = test_loss


Epoch: 01 | Epoch Time: 0m 1s
	Training Loss: 0.032834 \Test Loss: 0.029671
Test loss decreased (inf --> 0.029671). Saving model...
Epoch: 02 | Epoch Time: 0m 1s
	Training Loss: 0.023768 \Test Loss: 0.023491
Test loss decreased (0.029671 --> 0.023491). Saving model...
Epoch: 03 | Epoch Time: 0m 1s
	Training Loss: 0.020612 \Test Loss: 0.022441
Test loss decreased (0.023491 --> 0.022441). Saving model...
Epoch: 04 | Epoch Time: 0m 1s
	Training Loss: 0.019368 \Test Loss: 0.021934
Test loss decreased (0.022441 --> 0.021934). Saving model...
Epoch: 05 | Epoch Time: 0m 1s
	Training Loss: 0.018702 \Test Loss: 0.023604
Epoch: 06 | Epoch Time: 0m 1s
	Training Loss: 0.017162 \Test Loss: 0.024530
Epoch: 07 | Epoch Time: 0m 1s
	Training Loss: 0.016412 \Test Loss: 0.021022
Test loss decreased (0.021934 --> 0.021022). Saving model...
Epoch: 08 | Epoch Time: 0m 1s
	Training Loss: 0.015720 \Test Loss: 0.018891
Test loss decreased (0.021022 --> 0.018891). Saving model...
Epoch: 09 | Epoch Time: 0m 1s
	

In [27]:
y_pred_list = []
y_targ_list = []
model = RNNet(EMBEDDING_DIM, RNN_HIDDEN_DIM, OUTPUT_DIM).to(device)
model.load_state_dict(torch.load(RNN_MODEL_PATH))
model.eval()

with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())

y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]

print(classification_report(y_targ_list, y_pred_list))


              precision    recall  f1-score   support

           0       0.98      0.84      0.91      1677
           1       0.40      0.84      0.54       206

    accuracy                           0.84      1883
   macro avg       0.69      0.84      0.72      1883
weighted avg       0.91      0.84      0.87      1883



<span style="color:darkviolet">
<font size="5">Gated RNN</font><br>
</span>


In [28]:
class GRU_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(GRU_Network, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=1, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, self.hidden_dim))

        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out)
        return out


In [29]:
GRU_HIDDEN_DIM = 512
GRU_N_EPOCHS = 6


In [30]:
model = GRU_Network(EMBEDDING_DIM, GRU_HIDDEN_DIM, OUTPUT_DIM)
model = model.to(device)
loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight))
optimizer = optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(GRU_N_EPOCHS):

    start_time = time.time()
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        inputs = torch.squeeze(inputs)
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs)
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
        torch.save(model.state_dict(), GRU_MODEL_PATH)
        test_min_loss = test_loss

Epoch: 01 | Epoch Time: 0m 7s
	Training Loss: 0.026954 \Test Loss: 0.023842
Test loss decreased (inf --> 0.023842). Saving model...
Epoch: 02 | Epoch Time: 0m 6s
	Training Loss: 0.020906 \Test Loss: 0.022346
Test loss decreased (0.023842 --> 0.022346). Saving model...
Epoch: 03 | Epoch Time: 0m 7s
	Training Loss: 0.020220 \Test Loss: 0.021780
Test loss decreased (0.022346 --> 0.021780). Saving model...
Epoch: 04 | Epoch Time: 0m 6s
	Training Loss: 0.019413 \Test Loss: 0.021524
Test loss decreased (0.021780 --> 0.021524). Saving model...
Epoch: 05 | Epoch Time: 0m 7s
	Training Loss: 0.018809 \Test Loss: 0.023243
Epoch: 06 | Epoch Time: 0m 6s
	Training Loss: 0.018164 \Test Loss: 0.021354
Test loss decreased (0.021524 --> 0.021354). Saving model...


In [31]:
y_pred_list = []
y_targ_list = []
model = GRU_Network(EMBEDDING_DIM, GRU_HIDDEN_DIM, OUTPUT_DIM).to(device)
model.load_state_dict(torch.load(GRU_MODEL_PATH))
model.eval()

with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())

y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]

print(classification_report(y_targ_list, y_pred_list))

              precision    recall  f1-score   support

           0       0.97      0.83      0.89      1677
           1       0.36      0.76      0.49       206

    accuracy                           0.82      1883
   macro avg       0.66      0.80      0.69      1883
weighted avg       0.90      0.82      0.85      1883



<span style="color:darkviolet">
<font size="5">LSTM</font><br>
</span>


In [32]:
# LSTM


class LSTM_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(LSTM_Network, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, self.hidden_dim))
        c0 = Variable(torch.zeros(1, self.hidden_dim))

        # One time step
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out)

        return out


In [33]:
LSTM_HIDDEN_DIM = 20
LSTM_N_EPOCHS = 10

In [34]:
model = LSTM_Network(EMBEDDING_DIM, LSTM_HIDDEN_DIM, OUTPUT_DIM)
model = model.to(device)
loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight))
optimizer = optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(LSTM_N_EPOCHS):

    start_time = time.time()
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        inputs = torch.squeeze(inputs)
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs)
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
        torch.save(model.state_dict(), LSTM_MODEL_PATH)
        test_min_loss = test_loss

Epoch: 01 | Epoch Time: 0m 1s
	Training Loss: 0.032762 \Test Loss: 0.029273
Test loss decreased (inf --> 0.029273). Saving model...
Epoch: 02 | Epoch Time: 0m 1s
	Training Loss: 0.025426 \Test Loss: 0.024127
Test loss decreased (0.029273 --> 0.024127). Saving model...
Epoch: 03 | Epoch Time: 0m 1s
	Training Loss: 0.021819 \Test Loss: 0.022879
Test loss decreased (0.024127 --> 0.022879). Saving model...
Epoch: 04 | Epoch Time: 0m 1s
	Training Loss: 0.020454 \Test Loss: 0.022493
Test loss decreased (0.022879 --> 0.022493). Saving model...
Epoch: 05 | Epoch Time: 0m 1s
	Training Loss: 0.019655 \Test Loss: 0.022109
Test loss decreased (0.022493 --> 0.022109). Saving model...
Epoch: 06 | Epoch Time: 0m 1s
	Training Loss: 0.019404 \Test Loss: 0.021881
Test loss decreased (0.022109 --> 0.021881). Saving model...
Epoch: 07 | Epoch Time: 0m 1s
	Training Loss: 0.018814 \Test Loss: 0.022930
Epoch: 08 | Epoch Time: 0m 1s
	Training Loss: 0.018450 \Test Loss: 0.021499
Test loss decreased (0.021881 -

In [35]:
y_pred_list = []
y_targ_list = []
model = LSTM_Network(EMBEDDING_DIM, LSTM_HIDDEN_DIM, OUTPUT_DIM).to(device)
model.load_state_dict(torch.load(LSTM_MODEL_PATH))
model.eval()

with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())

y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]

print(classification_report(y_targ_list, y_pred_list))

              precision    recall  f1-score   support

           0       0.96      0.84      0.90      1677
           1       0.37      0.75      0.50       206

    accuracy                           0.83      1883
   macro avg       0.67      0.80      0.70      1883
weighted avg       0.90      0.83      0.86      1883



<span style="color:darkviolet">
<font size="5">Bi-LSTM</font><br>
</span>


In [36]:
# LSTM


class Bi_LSTM_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(Bi_LSTM_Network, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True, bidirectional=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1 * 2, self.hidden_dim))
        c0 = Variable(torch.zeros(1 * 2, self.hidden_dim))

        # One time step
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out)

        return out


In [37]:
BILSTM_HIDDEN_DIM = 20
BILSTM_N_EPOCHS = 10

In [38]:
model = Bi_LSTM_Network(EMBEDDING_DIM, BILSTM_HIDDEN_DIM, OUTPUT_DIM)
model = model.to(device)
loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight))
optimizer = optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(BILSTM_N_EPOCHS):

    start_time = time.time()
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        inputs = torch.squeeze(inputs)
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs)
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
        torch.save(model.state_dict(), BILSTM_MODEL_PATH)
        test_min_loss = test_loss

Epoch: 01 | Epoch Time: 0m 2s
	Training Loss: 0.031517 \Test Loss: 0.028087
Test loss decreased (inf --> 0.028087). Saving model...
Epoch: 02 | Epoch Time: 0m 2s
	Training Loss: 0.024170 \Test Loss: 0.023531
Test loss decreased (0.028087 --> 0.023531). Saving model...
Epoch: 03 | Epoch Time: 0m 2s
	Training Loss: 0.020926 \Test Loss: 0.023109
Test loss decreased (0.023531 --> 0.023109). Saving model...
Epoch: 04 | Epoch Time: 0m 2s
	Training Loss: 0.020131 \Test Loss: 0.023165
Epoch: 05 | Epoch Time: 0m 2s
	Training Loss: 0.019162 \Test Loss: 0.022367
Test loss decreased (0.023109 --> 0.022367). Saving model...
Epoch: 06 | Epoch Time: 0m 3s
	Training Loss: 0.019182 \Test Loss: 0.022559
Epoch: 07 | Epoch Time: 0m 2s
	Training Loss: 0.018238 \Test Loss: 0.023301
Epoch: 08 | Epoch Time: 0m 2s
	Training Loss: 0.017657 \Test Loss: 0.021603
Test loss decreased (0.022367 --> 0.021603). Saving model...
Epoch: 09 | Epoch Time: 0m 2s
	Training Loss: 0.017508 \Test Loss: 0.021044
Test loss decrea

In [39]:
y_pred_list = []
y_targ_list = []
model = Bi_LSTM_Network(EMBEDDING_DIM, BILSTM_HIDDEN_DIM, OUTPUT_DIM).to(device)
model.load_state_dict(torch.load(BILSTM_MODEL_PATH))
model.eval()

with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())

y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]

print(classification_report(y_targ_list, y_pred_list))

              precision    recall  f1-score   support

           0       0.97      0.82      0.88      1677
           1       0.34      0.78      0.47       206

    accuracy                           0.81      1883
   macro avg       0.65      0.80      0.68      1883
weighted avg       0.90      0.81      0.84      1883

